In [89]:
import sys
import os
# caution: path[0] is reserved for script path (or '' in REPL).
sys.path.insert(1, os.path.abspath('./../src'))

import spacepy.time
import numpy as np
import datetime
import pandas as pd

import data_loader
import data_references

import matplotlib.pyplot as plt
from matplotlib import colors

import importlib
importlib.reload(data_loader)
importlib.reload(data_references)

%matplotlib qt

In [83]:
start = datetime.datetime(year = 2013, month=3, day=1)
end = datetime.datetime(year = 2013, month=4, day=1)

metop1 = data_loader.load_compressed_poes_data("metop1", start=start, end=end)
metop2 = data_loader.load_compressed_poes_data("metop2", start=start, end=end)
noaa15 = data_loader.load_compressed_poes_data("noaa15", start=start, end=end)
noaa16 = data_loader.load_compressed_poes_data("noaa16", start=start, end=end)
noaa18 = data_loader.load_compressed_poes_data("noaa18", start=start, end=end)
noaa19 = data_loader.load_compressed_poes_data("noaa19", start=start, end=end)

satellites = [metop1, metop2, noaa15, noaa16, noaa18, noaa19]


Loading : POES_201303_metop1_CLEAN.npz
Loading : POES_201304_metop1_CLEAN.npz
Loading : POES_201303_metop2_CLEAN.npz
Loading : POES_201304_metop2_CLEAN.npz
Loading : POES_201303_noaa15_CLEAN.npz
Loading : POES_201304_noaa15_CLEAN.npz
Loading : POES_201303_noaa16_CLEAN.npz
Loading : POES_201304_noaa16_CLEAN.npz
Loading : POES_201303_noaa18_CLEAN.npz
Loading : POES_201304_noaa18_CLEAN.npz
Loading : POES_201303_noaa19_CLEAN.npz
Loading : POES_201304_noaa19_CLEAN.npz


In [138]:
#Parameters
L_bins = np.arange(3.0, 7.5, 0.25)
epoch_bin_size = "6h"


epoch_start = pd.Timestamp(start)
epoch_end = pd.Timestamp(end + pd.Timedelta(epoch_bin_size))
epoch_bins = pd.to_datetime(np.arange(epoch_start, epoch_end, pd.Timedelta(epoch_bin_size)))

total = pd.DataFrame({"Satid": [], "L" : [], "MLT" : [], "CHORUS": []})

for satellite in satellites:
        
    df = pd.DataFrame({"Timestamp" : satellite.get_data(data_references.POES.EPOCH),
                        'L': satellite.get_data(data_references.POES.L), 
                        'MLT' : satellite.get_data(data_references.POES.MLT), 
                        "CHORUS" : satellite.get_data(data_references.POES.NAIVE_CHORUS_AMPLITUDES),
                        "Satid" : satellite.satid})
    
    total = pd.concat([total, df], ignore_index=True, sort=False)

        

total = total.drop(total[total["L"] <= min(L_bins)].index)
total = total.drop(total[total["L"] > max(L_bins)].index)
 
 

total["L_BIN"] = pd.cut(total["L"], L_bins, labels=False)
total["DATE_BIN"] = pd.cut(total["Timestamp"], epoch_bins, labels=False)
    
group_df = total.groupby(["L_BIN", "DATE_BIN"], dropna=True)



C:\Users\brofe\AppData\Local\Temp\ipykernel_24632\3568007584.py:20: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  total = pd.concat([total, df], ignore_index=True, sort=False)


In [136]:
averages = np.zeros((len(L_bins) - 1, len(epoch_bins) - 1))

for i in range(averages.shape[0]):
    for j in range(averages.shape[1]):
        
        averages[i, j] = group_df[["CHORUS"]].get_group((i, j)).mean()["CHORUS"]


averages = averages + 1e-10 #Stabilize the log norm

fig, ax = plt.subplots(1, 1, figsize=(16, 9), sharex=True)

image_cmap = ax.imshow(averages, 
                       origin="lower", 
                       extent=[epoch_bins[0], epoch_bins[-1], L_bins[0], L_bins[-1]],
                       norm=colors.LogNorm(vmin=1, vmax=100),
                       aspect="auto",
                       interpolation="none")


cbar = plt.colorbar(image_cmap, ax=ax, pad=0.01)

cbar.set_label("Bw (pT)\n", loc="center", labelpad=15, rotation=270)

ax.set_title(f"Chorus: {start.year} / {start.month}")
plt.tight_layout()